In [ ]:
import pandas as pd
import requests
import json
import urllib.request
import time
from bs4 import BeautifulSoup
import re
import numpy as np

In [ ]:
pip install rank_bm25

In [ ]:
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
Whole_dataset=pd.read_csv(r"C:\Users\sigma\Desktop\Saner\Dataset (Primarry)\mobile_preproccessed_whole_dataset.csv")
Whole_dataset

,Unnamed: 0,Bug ID,Type,Summary,Product,Component,Status,Resolution,Updated,Bug Id,Description,Duplicate_Bug_Ids
0,0,1668376,defect,Test failure in dom/base/test/test_window_clos...,GeckoView,General,RESOLVED,FIXED,2020-10-14 03:01:38,1668376,the symptom was that the broadcastchannel me...,NaN
1,1,1631754,defect,PanZoomController erroneously returns INPUT_RE...,GeckoView,General,RESOLVED,FIXED,2020-05-23 04:47:46,1631754,for websites which have their own event handle...,NaN
2,2,1677838,defect,NumberFormatExceptionjava.lang.Integer in pars...,GeckoView,General,RESOLVED,FIXED,2021-01-06 13:28:48,1677838,we are seeing these crash being reported on se...,NaN
3,4,1686100,enhancement,Route Service.onTrimMemory notifications throu...,GeckoView,General,RESOLVED,FIXED,2021-01-12 02:17:23,1686100,since android may send these to the services h...,NaN
4,5,1630229,defect,Crash in [@ java.lang.AssertionError: at org.m...,GeckoView,General,RESOLVED,FIXED,2020-05-28 05:17:09,1630229,this bug is for crash report bp ebbf ...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5315,5499,1683643,defect,Intermittent org.mozilla.geckoview.test.Select...,GeckoView,General,RESOLVED,DUPLICATE,2021-07-30 19:04:17,1683643,filed by rmaries at mozilla com parse...,1682775.0
5316,5500,1649160,defect,Download Feature: unicode RTLO char can fake t...,Firefox for iOS,General,RESOLVED,FIXED,2020-12-18 17:20:48,1649160,created attachment regedt‮gpj exe summary ...,NaN
5317,5501,1514619,defect,Menu options are in different order than new t...,Firefox for iOS,Menu and Toolbar,RESOLVED,INVALID,2019-02-11 16:36:05,1514619,created attachment png the hostor...,NaN
5318,5502,1546822,defect,Notifications repeating if not acted on in iOS...,Firefox for iOS,Sync,RESOLVED,WORKSFORME,2019-09-18 10:10:20,1546822,created attachment syncnotificationecho png ...,NaN


In [ ]:
data = Whole_dataset.copy()

In [ ]:
data['Description1'] = data['Summary']+ data['Description']

In [ ]:
data.info()

In [ ]:
data = data[["Bug ID", "Description1", 'Duplicate_Bug_Ids', 'Resolution']]

In [ ]:
data

In [ ]:
data = data.rename(columns = {"Bug ID":"Bug_Id", "Description1": "Description"})
data

In [ ]:
data['Bug_Id']= data['Bug_Id'].values.astype('float64')

In [ ]:
sentences = data['Description']

In [ ]:
tokens = []
for s in sentences:
    cleanedTex=re.sub(r'[^\w\s]','',str(s)).lower()
    words = (word_tokenize(cleanedTex))
    tokens .append(words)

In [ ]:
bm25 = BM25Okapi(tokens)

In [ ]:
nan_value = float("NaN")
data.replace("", nan_value, inplace=True)
data.dropna(subset = ["Description"], inplace=True)
cond = (data['Description'] == 'NaN')
cond.unique()
data = data.reset_index(drop=True)

In [ ]:
data['Description']

In [ ]:
data['Description']=data['Description'].values.astype('object')

### On complete data

In [ ]:
count=0
dup_indices_with_bug_id = {}
similarity_scores_before = []
similarity_scores = {}
issues = data.Bug_Id

for i, row in data.iterrows():
    
    issue_id = row.Bug_Id
    dup_id = row.Duplicate_Bug_Ids
    
    if dup_id not in issues.values:
        count+=1
        continue

    similarity_row = bm25.get_scores(tokens[i])
    argsort_similarity = np.argsort(similarity_row)[::-1][1:]
    similarity_scores_before = (np.sort(similarity_row)[::-1][1:])
    dup_bug_index = list(issues[issues == dup_id].index)[0]
    dup_ranking = np.argmax(argsort_similarity == np.int64(dup_bug_index))
    
    dup_indices_with_bug_id[issue_id] = dup_ranking

In [ ]:
count_at_1=0
count_at_5=0
count_at_10=0
count_at_20=0
count_at_25=0
count_at_30=0
count_at_50=0
count_at_75=0
count_at_100=0

for key, value in enumerate(dup_indices_with_bug_id.values()):
    if(value < 1):
        count_at_1+=1
    if(value < 5):
        count_at_5+=1
    if(value < 10):
        count_at_10+=1
    if(value < 20):
        count_at_20+=1
    if(value < 25):
        count_at_25+=1
    if(value < 30):
        count_at_30+=1
    if(value < 50):
        count_at_50+=1
    if(value < 75):
        count_at_75+=1
    if(value < 100):
        count_at_100+=1
        
print("count at 1",count_at_1)        
print("count at 5",count_at_5)
print("count at 10",count_at_10)
print("count at 20",count_at_20)
print("count at 25",count_at_25)
print("count at 30",count_at_30)
print("count at 50",count_at_50)
print("count at 75",count_at_75)
print("count at 100",count_at_100)

N = len(dup_indices_with_bug_id)

recall_rate_at_1 = count_at_1/N
print("recall_rate_at_1",recall_rate_at_1)

recall_rate_at_5 = count_at_5/N
print("recall_rate_at_5",recall_rate_at_5)

recall_rate_at_10 = count_at_10/N
print("recall_rate_at_10",recall_rate_at_10)

recall_rate_at_20 = count_at_20/N
print("recall_rate_at_20",recall_rate_at_20)

recall_rate_at_25 = count_at_25/N
print("recall_rate_at_25",recall_rate_at_25)

recall_rate_at_30 = count_at_30/N
print("recall_rate_at_30",recall_rate_at_30)

recall_rate_at_50 = count_at_50/N
print("recall_rate_at_50",recall_rate_at_50)

recall_rate_at_75 = count_at_75/N
print("recall_rate_at_75",recall_rate_at_75)

recall_rate_at_100 = count_at_100/N
print("recall_rate_at_10",recall_rate_at_100)

## Textually Similar

In [ ]:
textually_similar_data = pd.read_csv(r"C:\Users\sigma\Desktop\Saner\Mobile\Mobile_final_sim.csv")

In [ ]:
textually_similar_data['Description1'] = textually_similar_data['Summary']+ textually_similar_data['Description']

In [ ]:
textually_similar_data = textually_similar_data.rename(columns = {"Bug ID":"Bug_Id"})

In [ ]:
textually_similar_data = textually_similar_data[["Bug_Id", "Description1", 'Duplicate_Bug_Ids', 'Resolution']]

In [ ]:
textually_similar_data

In [ ]:
bm25 = BM25Okapi(tokens)

In [ ]:
nan_value = float("NaN")
textually_similar_data.replace("", nan_value, inplace=True)
textually_similar_data.dropna(subset = ["Description1"], inplace=True)
cond = (textually_similar_data['Description1'] == 'NaN')
cond.unique()
textually_similar_data = textually_similar_data.reset_index(drop=True)

In [ ]:
textually_similar_data['Description1']=textually_similar_data['Description1'].values.astype('object')

In [ ]:
textually_similar_data['Bug_Id']= textually_similar_data['Bug_Id'].values.astype('float64')

In [ ]:
sentence_sim = textually_similar_data.Description1

tokens_sim = []
for s in sentence_sim:
    cleanedTex=re.sub(r'[^\w\s]','',str(s)).lower()
    words = (word_tokenize(cleanedTex))
    tokens_sim.append(words)

In [ ]:
count=0
dup_indices_with_bug_id_similar = {}
similarity_scores_before = []
similarity_scores = {}
issues = data.Bug_Id

for i, row in textually_similar_data.iterrows():
    
    issue_id = row.Bug_Id
    dup_id = row.Duplicate_Bug_Ids
    
    if dup_id not in issues.values:
        count+=1
        continue
        
    similarity_row = bm25.get_scores(tokens_sim[i])
    argsort_similarity = np.argsort(similarity_row)[::-1][1:]
    similarity_scores_before = (np.sort(similarity_row)[::-1][1:])
    dup_bug_index = list(issues[issues == dup_id].index)[0]
    dup_ranking = np.argmax(argsort_similarity == np.int64(dup_bug_index)) 
    dup_indices_with_bug_id_similar[issue_id] = dup_ranking

In [ ]:
count_at_1=0
count_at_5=0
count_at_10=0
count_at_20=0
count_at_25=0
count_at_30=0
count_at_50=0
count_at_75=0
count_at_100=0

for key, value in enumerate(dup_indices_with_bug_id_similar.values()):
    if(value < 1):
        count_at_1+=1
    if(value < 5):
        count_at_5+=1
    if(value < 10):
        count_at_10+=1
    if(value < 20):
        count_at_20+=1
    if(value < 25):
        count_at_25+=1
    if(value < 30):
        count_at_30+=1
    if(value < 50):
        count_at_50+=1
    if(value < 75):
        count_at_75+=1
    if(value < 100):
        count_at_100+=1
        
print("count at 1",count_at_1)        
print("count at 5",count_at_5)
print("count at 10",count_at_10)
print("count at 20",count_at_20)
print("count at 25",count_at_25)
print("count at 30",count_at_30)
print("count at 50",count_at_50)
print("count at 75",count_at_75)
print("count at 100",count_at_100)

N = len(dup_indices_with_bug_id_similar)

recall_rate_at_1 = count_at_1/N
print("recall_rate_at_1",recall_rate_at_1)

recall_rate_at_5 = count_at_5/N
print("recall_rate_at_5",recall_rate_at_5)

recall_rate_at_10 = count_at_10/N
print("recall_rate_at_10",recall_rate_at_10)

recall_rate_at_20 = count_at_20/N
print("recall_rate_at_20",recall_rate_at_20)

recall_rate_at_25 = count_at_25/N
print("recall_rate_at_25",recall_rate_at_25)

recall_rate_at_30 = count_at_30/N
print("recall_rate_at_30",recall_rate_at_30)

recall_rate_at_50 = count_at_50/N
print("recall_rate_at_50",recall_rate_at_50)

recall_rate_at_75 = count_at_75/N
print("recall_rate_at_75",recall_rate_at_75)

recall_rate_at_100 = count_at_100/N
print("recall_rate_at_100",recall_rate_at_100)

# Textually Dissimilar

In [ ]:
textually_dissimilar_data = pd.read_csv(r"C:\Users\sigma\Desktop\Saner\Mobile\Mobile_final_dis.csv")

In [ ]:
textually_dissimilar_data.info()

In [ ]:
textually_dissimilar_data['Description1'] = textually_dissimilar_data['Summary']+ textually_dissimilar_data['Description']

In [ ]:
textually_dissimilar_data = textually_dissimilar_data.rename(columns = {"Bug ID":"Bug_Id"})

In [ ]:
textually_dissimilar_data = textually_dissimilar_data[["Bug_Id", "Description1", 'Duplicate_Bug_Ids', 'Resolution']]

In [ ]:
bm25 = BM25Okapi(tokens)

In [ ]:
nan_value = float("NaN")
textually_dissimilar_data.replace("", nan_value, inplace=True)
textually_dissimilar_data.dropna(subset = ["Description1"], inplace=True)
cond = (textually_dissimilar_data['Description1'] == 'NaN')
cond.unique()
textually_dissimilar_data = textually_dissimilar_data.reset_index(drop=True)

In [ ]:
textually_dissimilar_data['Description1']=textually_dissimilar_data['Description1'].values.astype('object')

In [ ]:
textually_dissimilar_data['Bug_Id']=textually_dissimilar_data['Bug_Id'].values.astype('float64')

In [ ]:
textually_dissimilar_data = textually_dissimilar_data[:1414]
textually_dissimilar_data

In [ ]:
sentence_dissim = textually_dissimilar_data.Description1

tokens_dissim = []
for s in sentence_dissim:
    cleanedTex=re.sub(r'[^\w\s]','',str(s)).lower()
    words = (word_tokenize(cleanedTex))
    tokens_dissim.append(words)

In [ ]:
count=0
dup_indices_with_bug_id_dissimilar = {}
similarity_scores_before = []
similarity_scores = {}
issues = data.Bug_Id

for i, row in textually_dissimilar_data.iterrows():
    
    issue_id = row.Bug_Id
    dup_id = row.Duplicate_Bug_Ids
    
    if dup_id not in issues.values:
        count+=1
#         print(f'{i}. Duplicate ID {dup_id} not found in issue IDs')
        continue
    if i==4427:
        break
    similarity_row = bm25.get_scores(tokens_dissim[i])
    argsort_similarity = np.argsort(similarity_row)[::-1][1:]
    similarity_scores_before = (np.sort(similarity_row)[::-1][1:])
#     similarity_scores[issue_id] = np.min(similarity_scores_before[np.nonzero(similarity_scores_before)])
    dup_bug_index = list(issues[issues == dup_id].index)[0]
    dup_ranking = np.argmax(argsort_similarity == np.int64(dup_bug_index))
    
    dup_indices_with_bug_id_dissimilar[issue_id] = dup_ranking
# print(count)

In [ ]:
count_at_1=0
count_at_5=0
count_at_10=0
count_at_20=0
count_at_25=0
count_at_30=0
count_at_50=0
count_at_75=0
count_at_100=0

for key, value in enumerate(dup_indices_with_bug_id_dissimilar.values()):
    if(value < 1):
        count_at_1+=1
    if(value < 5):
        count_at_5+=1
    if(value < 10):
        count_at_10+=1
    if(value < 20):
        count_at_20+=1
    if(value < 25):
        count_at_25+=1
    if(value < 30):
        count_at_30+=1
    if(value < 50):
        count_at_50+=1
    if(value < 75):
        count_at_75+=1
    if(value < 100):
        count_at_100+=1
        
print("count at 1",count_at_1)        
print("count at 5",count_at_5)
print("count at 10",count_at_10)
print("count at 20",count_at_20)
print("count at 25",count_at_25)
print("count at 30",count_at_30)
print("count at 50",count_at_50)
print("count at 75",count_at_75)
print("count at 100",count_at_100)

N = len(dup_indices_with_bug_id_dissimilar)

recall_rate_at_1 = count_at_1/N
print("recall_rate_at_1",recall_rate_at_1)

recall_rate_at_5 = count_at_5/N
print("recall_rate_at_5",recall_rate_at_5)

recall_rate_at_10 = count_at_10/N
print("recall_rate_at_10",recall_rate_at_10)

recall_rate_at_20 = count_at_20/N
print("recall_rate_at_20",recall_rate_at_20)

recall_rate_at_25 = count_at_25/N
print("recall_rate_at_25",recall_rate_at_25)

recall_rate_at_30 = count_at_30/N
print("recall_rate_at_30",recall_rate_at_30)

recall_rate_at_50 = count_at_50/N
print("recall_rate_at_50",recall_rate_at_50)

recall_rate_at_75 = count_at_75/N
print("recall_rate_at_75",recall_rate_at_75)

recall_rate_at_100 = count_at_100/N
print("recall_rate_at_100",recall_rate_at_100)